In [1]:
from collections import namedtuple
# neptune
import os
import numpy as np

from time import sleep
import torch
import pytorch_lightning as pl
from torch.nn import functional as F
from transformers import  T5ForConditionalGeneration, T5Config, get_linear_schedule_with_warmup, AdamW
from project.data.data import *

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gersa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Data processing

In [2]:
run_preprocessing = False
# No need to rerun the processing

if run_preprocessing:
    processor = RaceDataProcessor()
    processor.process_data("RACE/", "Processed_New/")
else:
    print("Data Processing: skip")

Data Processing: skip


In [3]:
class Args:
    data_path =  "Processed_New/"
    batch_size =  16
    num_workers =  0
    pretrained_model = "t5-base"
    tokenizer_len = 0
    weights_decay = 1e-5
    learning_rate =1e-5

hparams=Args()

In [5]:
data_module = RaceDataModule(hparams)
## Colab is weird, so  I have to put it here
#data_module.tokenizer.add_special_tokens({'additional_special_tokens': ['<answer>', '<context>']})
hparams.tokenizer_len = len(data_module.tokenizer)

data_module.prepare_data()
data_module.setup()

SETUP: Training Dataset
SETUP: Validation Dataset
SETUP: Test Dataset


In [ ]:
model = T5FinetuneForRACE(hparams)

trainer = 

In [4]:
class T5FinetuneForRACE(pl.LightningModule):
    def __init__(self, hparams):
        super(T5FinetuneForRACE, self).__init__()
        self.save_hyperparameters()
        self.hparams = hparams
        if self.hparams.pretrained_model == "t5-base":
            config = T5Config(decoder_start_token_id=0)
            self.model = T5ForConditionalGeneration(config).from_pretrained(self.hparams.pretrained_model)
            self.model.resize_token_embeddings(self.hparams.tokenizer_len)
        else:
            raise NotImplementedError
 
    def forward(self, ids, mask, y_ids, lm_labels):
        return model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, lm_labels=lm_labels)


    def training_step(self, batch, batch_idx):
        input_ids, attention_mask, decoder_input_ids, lm_labels = batch
        y_hat = self(input_ids, attention_mask, decoder_input_ids, lm_labels)
        loss = F.cross_entropy(y_hat[0], y)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log('valid_loss', loss)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log('test_loss', loss)

    def configure_optimizers(self):
        no_decay = ['bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [{'params': [p for n, p in self.named_parameters() if not any(nd in n for nd in no_decay)],
                                         'weight_decay': 0.01}, 
                                        {'params': [p for n, p in self.named_parameters() if any(nd in n for nd in no_decay)],
                                         'weight_decay': 0.0}]
        
        optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr = self.hparams.learning_rate)
#         scheduler = get_linear_schedule_with_warmup(
#             optimizer,
#             num_warmup_steps=0,
#             # Default value in run_glue.py
#             num_training_steps=self.hparams.num_training_steps)
 
        return [optimizer], [scheduler]

In [7]:
len([32100,   432,     8,   756,     5, 32101,   932,   305,    19,  4318,
          4351,    31,     7,  1430,     5,    86,   685,     6,    34,    19,
          1086,   718,  7508,    31,     7,  1430,   250,    34,    19,     3,
          4894,  9443,    41,     3,    61,    57,  5234,     5,    37,  5216,
            13,  7508,    31,     7,  1430,    65,     3,     9,   307,   892,
             5,    94,    19,   243,    24,     8,  3994,   639,    45,     8,
         10282, 17733,  3397,    16,  1473,     5,   461,  7508,    31,     7,
          1430,     6,  4318,  5234,     3,  1544,     3,     9,   534,   773,
            13,  6605,  4340,     5,    94,    19,  2303,    28,     3,     9,
          8384,    11,  3353,    28, 15431, 11388,    41,     3,   137,  4351,
           333,    12,     3,  1544,    34,   182,   231,     5,   461,    48,
           239,     6,   502,   278,    31,    17,    43,    12,   281,    12,
           496,     5,  1698,   384,    28,     3,     9,  4940,  5168,     7,
            95,  1450,   443,   102,    18,  6489,  5692,    41,     3,   137,
            37,  5692,     7,    33,  1086,   386,  2602,    10,  1001,     6,
          1131,    11,  1692,     5,  1589,  5024,    21,     8,  2353,     6,
          1131,  5024,    21,     8,  2039,    11,  1692,  5024,    21,     8,
           520,     5,  4318,  1362,   857,    24,     8,  5692,     7,    56,
           830,    70,  5234,   207,  5851,    11,   428,   135, 11578,    11,
           579,     5,     1])

193

In [14]:
[data_module.tokenizer.decode(t) for t in i["articles"]["input_ids"]]

["<answer> All the above.<context> May 5 is Japanese Children's Day. In fact, it is usually called Boy's Day because it is mainly celebrated ( ) by boys. The celebration of Boy's Day has a long history. It is said that the festival comes from the Dragon Boat Festival in China. On Boy's Day, Japanese boys eat a special kind of rice cake. It is covered with a leaf and filled with bean paste ( ). Children love to eat it very much. On this day, children don't have to go to school. Each family with a boy hangs up huge carp-shaped flag ( ). The flags are usually three colors: black, red and blue. Black stands for the father, red stands for the mother and blue stands for the son. Japanese parents believe that the flags will bring their boys good luck and give them courage and power.</s>"]

In [6]:
for i in data_module.train_dataloader():
    print(i)
    break

COLLATE based of t5-base
{'articles': {'input_ids': tensor([[32100,   432,     8,   756,     5, 32101,   932,   305,    19,  4318,
          4351,    31,     7,  1430,     5,    86,   685,     6,    34,    19,
          1086,   718,  7508,    31,     7,  1430,   250,    34,    19,     3,
          4894,  9443,    41,     3,    61,    57,  5234,     5,    37,  5216,
            13,  7508,    31,     7,  1430,    65,     3,     9,   307,   892,
             5,    94,    19,   243,    24,     8,  3994,   639,    45,     8,
         10282, 17733,  3397,    16,  1473,     5,   461,  7508,    31,     7,
          1430,     6,  4318,  5234,     3,  1544,     3,     9,   534,   773,
            13,  6605,  4340,     5,    94,    19,  2303,    28,     3,     9,
          8384,    11,  3353,    28, 15431, 11388,    41,     3,   137,  4351,
           333,    12,     3,  1544,    34,   182,   231,     5,   461,    48,
           239,     6,   502,   278,    31,    17,    43,    12,   281,    12,


In [32]:
next(data_module.train_dataloader())

TypeError: 'DataLoader' object is not an iterator

In [16]:
data_module.train_dataloader().collate_fn(i)

COLLATE based of t5-base


TypeError: string indices must be integers